In [1]:
import pandas as pd
from gnb import GNB

train_filepath = 'data/iris_train_set.csv'
test_filepath = 'data/iris_test_set.csv'

train_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)



In [2]:
ytrain = train_data['iris_type']
Xtrain = train_data.drop(['iris_type'], axis=1)
ytest = test_data['iris_type']
Xtest = test_data.drop(['iris_type'], axis=1)

print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(120, 4)
(120,)
(30, 4)
(30,)


In [3]:
gnb = GNB()

In [4]:
gnb.fit(Xtrain, ytrain)

In [5]:
ypred = gnb.predict(Xtest)

In [6]:
gnb.calc_accuracy(ypred, ytest)

1.0

In [7]:
from knn import KNN
knn = KNN(3,'euclidean','t-disribution')

In [8]:
knn.fit(Xtrain, ytrain)

In [9]:
pred = knn.predict(Xtest)

In [10]:
knn.calc_accuracy(ypred, ytest)

1.0

In [11]:
from kmeans import KMeans
kmeans = KMeans(3,init='kmeans++',scaling='minmax', change_val = 0.0001)

In [12]:
kmeans.fit(Xtrain.loc[:,['sepal length (cm)', 'petal length (cm)']])

[(0, array([0.58823529, 0.77966102]))]
[0.58823529 0.77966102] [0.5       0.6779661]
[0.58823529 0.77966102] [0.61764706 0.77966102]
[0.58823529 0.77966102] [1.         0.86440678]
[0.58823529 0.77966102] [0.52941176 0.61016949]
[0.58823529 0.77966102] [0.73529412 0.76271186]
[0.58823529 0.77966102] [0.52941176 0.50847458]
[0.58823529 0.77966102] [0.58823529 0.66101695]
[0.58823529 0.77966102] [0.5        0.59322034]
[0.58823529 0.77966102] [0.64705882 0.71186441]
[0.58823529 0.77966102] [0.73529412 0.83050847]
[0.58823529 0.77966102] [0.52941176 0.66101695]
[0.58823529 0.77966102] [0.23529412 0.08474576]
[0.58823529 0.77966102] [0.02941176 0.05084746]
[0.58823529 0.77966102] [0.20588235 0.38983051]
[0.58823529 0.77966102] [0.23529412 0.11864407]
[0.58823529 0.77966102] [0.58823529 0.69491525]
[0.58823529 0.77966102] [0.14705882 0.10169492]
[0.58823529 0.77966102] [0.5        0.69491525]
[0.58823529 0.77966102] [0.17647059 0.06779661]
[0.58823529 0.77966102] [0.05882353 0.05084746]
[0.

IndexError: invalid index to scalar variable.

In [ ]:
pred = kmeans.predict(Xtest.loc[:,['sepal length (cm)', 'petal length (cm)']])
pred

In [ ]:
kmeans.silhouette_score()

In [ ]:
import seaborn as sns



In [ ]:
f = sns.scatterplot(data = Xtest, x='sepal length (cm)', y='petal length (cm)', c=pred)

In [ ]:
f = sns.scatterplot(data = Xtrain, x='sepal length (cm)', y='petal length (cm)',c=kmeans.pred)

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, y_true = make_blobs(n_samples=500, centers=6,
                       cluster_std=0.60, random_state=0)


In [ ]:
X = pd.DataFrame(X)
X

In [ ]:
clf = KMeans(6)
clf.fit(X)
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=50,c=clf.pred)

In [ ]:
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=50,c=y_true)